# PostgreSQL

In [1]:
import sqlalchemy as db
import zeit
import pandas as pd
import configparser

In [2]:
c = configparser.ConfigParser()
c.read("config.ini")
zeit_key = c.get("Zeit", "key")
uri = c.get("Postgres", "URI")

#set the API, authenticate and check status
#api = zeit.API()
#api.set_token(zeit_key)
#api.get_status()

In [4]:
engine = db.create_engine("postgresql://code@localhost/test") #local
#engine = db.create_engine(uri) #on herokus
con = engine.connect()
metadata = db.MetaData()

### Retrieving Values

In [5]:
table = db.Table("keyword", metadata, autoload=True, autoload_with=engine)

In [6]:
word = "Merkel"
query = db.select([table])#.where(table.columns.uri == f"http://api.zeit.de/keyword/{word}")
ResultProxy = con.execute(query)
result = ResultProxy.fetchmany()
try:
    df = pd.DataFrame(result, columns = result[0].keys())
except Exception:
    print("No Results")

In [18]:
df.to_json("../data/startup.json")

In [14]:
pd.read_json(str(st), index_column = "uri")

TypeError: read_json() got an unexpected keyword argument 'index_column'

### Inserting Values

In [17]:
table.columns.keys()

['uri', 'lexical', 'results', 'type', 'json', 'matches', 'score']

In [35]:
r = api.get_keyword("armin-laschet", facet_time="1year")

In [36]:
query = db.insert(table).values(
    uri = r.uri, lexical = r.name, results = r.found, 
    type = r.type, json = r.get_facets().to_json(),
    matches = r.get_raw()["matches"][0], score = r.score
)
ResultProxy = con.execute(query)

### Deleting Values

In [32]:
delete = table.delete().where(table.columns.lexical == 'Laschet, Armin')
ResultProxy = con.execute(delete)

## Workflow for inserting

In [25]:
word_list = ["angela-merkel", "roettgen", "gazastreifen"]
for w in word_list:
    #call the api
    r = api.get_keyword(w, facet_time="1year", limit = 1)

    try:
        matches = r.get_raw()["matches"][0]
    except Exception:
        matches = ''

    #construct the insert query
    query = db.insert(table).values(
        uri = r.uri, lexical = r.name, results = r.found, 
        type = r.type, json = r.get_facets().to_json(),
        matches = matches, score = r.score,
        )
    #execute it
    ResultProxy = con.execute(query)

# Get the Data in Dash

In [ ]:
word = "gazastreifen"
query = db.select([table]).where(table.columns.uri == f"http://api.zeit.de/keyword/{word}")
ResultProxy = con.execute(query)
result = ResultProxy.fetchall()

In [ ]:
#better querys
query = db.select([table]).filter(db.or_( #SELECT * FROM table WHERE 
        table.columns.uri.like(f"%meck%"), #uri LIKE '%word%' OR
        table.columns.lexical.like(f"%meck%"), #lexical LIKE '%word%' OR
        )
        ).order_by(db.desc(table.columns.results)).limit(5) #ORDER BY results DESC LIMIT 5
ResultProxy = con.execute(query)
result = ResultProxy.fetchall()